In [38]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


def pivotize(m):
    n = len(m)
    ID = [[float(i == j) for i in range(n)] for j in range(n)]
    for j in range(n):
        row = max(range(j, n), key=lambda i: abs(m[i][j]))
        if j != row:
            ID[j], ID[row] = ID[row], ID[j]
    return ID


def SolveSystem(A, b):
    L, U, P = LUP(A)
    b = np.matmul(P,b)
    n = A.shape[0]
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - sum(L[i][k]*y[k] for k in range(i))
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        x[i] = (y[i] - sum(U[i][k]*x[k] for k in range(i+1,n)))/U[i][i]
    return x
        
    
    
    
def LUP(A):
    n = A.shape[0]
    L = np.zeros((n,n))
    U = np.zeros((n,n))
    P = pivotize(A)
    A2 = np.matmul(P, A)
    for j in range(n):
        L[j][j] = 1
        for i in range(j+1):
            s1 = sum(U[k][j] * L[i][k] for k in range(i))
            U[i][j] = A2[i][j] - s1
        for i in range(j, n):
            s2 = sum(U[k][j] * L[i][k] for k in range(j))
            L[i][j] = (A2[i][j] - s2) / U[j][j]
   
    return L, U, P


def RectangleMethod(x, xi, K, h):
    n = len(x)
    coeff = np.zeros(n)
    for i in range(n-1):
        coeff[i] = h * K(xi, (x[i] + x[i+1])/2)
    return coeff

def TrapezoidMethod(x, xi, K, h):
    n = len(x)
    coeff = np.zeros(n)
    coeff[0] = h/2 * K(xi, x[0])
    coeff[n-1] = h/2 * K(xi, x[n-1])
    for i in range(1,n-1):
        coeff[i] = h * K(xi, x[i])
    return coeff

def SimpsonMethod(x, xi, K, h):
    n = len(x)
    coeff = np.zeros(n)
    coeff[0] = h/3 * K(xi, x[0])
    coeff[n-1] = h/3 * K(xi, x[n-1])
    for i in range(1,n-1):
        coeff[i] = h/3 * K(xi, x[i]) * (4 if i % 2 == 1 else 2)
    return coeff




def FredgolmSolve(quadFun, h, lmd, K, f, rngx,  kind):
    x = [x for x in np.arange(rngx[0], rngx[1] + h, h)]
    n = len(x)
    coeff = np.zeros((n,n))
    fi = np.zeros(n)
    for i in range(n):
        coeff[i] =  lmd * quadFun(x, x[i], K, h)
        if (kind == 2):
            coeff[i][i] += 1
        fi[i] = f(x[i])
    return x, SolveSystem(coeff, fi) 


exactSolve = lambda x: x
K = lambda x,s: x * s
f = lambda x: 5/6 * x
lmd = -1/2
rngx = [0,1]
h = 0.01
kind = 2
xi, ui =  FredgolmSolve(SimpsonMethod,h, lmd, K, f, rngx, kind)
yi = np.array([exactSolve(xi[i]) for i in range(len(xi))]  )           
trace0 = go.Scatter(
    x=xi,
    y=ui,
    mode='markers',
    name='Approx solve',
    marker = dict(
        size = 4)
 
)
trace1 = go.Scatter(
    x=xi ,
    y=yi,
    name='Exact solve',
    mode = 'lines'
)


data = [trace1, trace0]

py.iplot(data, filename = 'basic-line')
    


High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~Coppelius/0 or inside your plot.ly account where it is named 'basic-line'


In [42]:
trace1 = go.Table(
    header=dict(values=['k', 'x', 'Approximate solution','Exact solution', 'eps compare with exact solution' ]),
    cells=dict(values=[[i for i in range(len(xi))],
                       xi,
                       ui,
                       yi,
                       abs(yi - ui)]))
                       


layout1 = dict(width=950, height=800, title='Eiler method')
fig = dict(data=[trace1], layout = layout1 )
py.iplot(fig, filename='table')